In [1]:
import pandas as pd


In [2]:
property_df = pd.read_csv('../data/curated/full_data.csv')

In [4]:
property_df.head()

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.000000,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.000000,3,2,1,House,spaciou home east,207021424,145.150927,-37.777456,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424.0
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.000000,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,-37.775254,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424.0
3,https://www.domain.com.au/7-davy-court-narre-w...,7 Davy Court Narre Warren South VIC 3805,795.000000,4,2,2,House,bedroom famili home prime locat warren south,212031457,145.318043,-38.061485,767.0,430.0,2350.0,2352.0,10833.0,Narre Warren South - East,>50.0,859.0,212031457.0
4,https://www.domain.com.au/level-1-301-13-15-la...,"Level 1, 301/13-15 Lake Street Caroline Spring...",793.269231,4,3,2,Apartment / Unit / Flat,endless possibl,213041463,144.744689,-37.731646,813.0,401.0,2277.0,2133.0,16696.0,Caroline Springs,>50.0,859.0,213041463.0
